In [1]:
import pandas as pd

In [2]:
# Load dataset
url = "assets/train_set_0520.csv"

In [3]:
#define column lables
names = ['id', 'type', 'path_to_file', 'begin_line', 'link_to_comment', 'comment', 'non-information']

In [4]:
#na_filter = false, because pandas read empty string as float (nan)!
#skiprows = 1, because the first row contains only the labels
dataframe = pd.read_csv(url, names=names, skiprows = 1, na_filter = False)

In [5]:
def get_java_file_name(v):
	x = v.split('/')
	y = x[-1].split('#')
	javaFileName = y[0]
	return javaFileName

def get_comment_line(v):
	x = v.split('/')
	y = x[-1].split('#')
	line = y[1][1:]
	return int(line)

def get_code_from_javadoc_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        if not (comment.startswith('/**') and comment.endswith('*/')):
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
        firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

def get_code_from_line_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        isComment = comment.startswith('//')
        if not isComment:
            code = comment.split('//')
            firstCodeLine = code[0].strip()
        else:
            while isComment:
                if 0 <= commentStartLine + i < len(lines):
                    comment = lines[commentStartLine + i].strip()
                    isComment = comment.startswith('//')
                    if isComment:
                        i += 1
                else:
                    isComment = False
            if 0 <= commentStartLine + i < len(lines):
                firstCodeLine = lines[commentStartLine + i].strip()
            else:
                firstCodeLine = ''
    return firstCodeLine 
    
def get_code_from_block_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        startsWithComment = comment.startswith('/*')
        if not startsWithComment:
            code = comment.split('/*')
            firstCodeLine = code[0].strip()
        else:
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
            firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

## extract One line code after comment
## Outfile: Paht to output CSV
## Javadir: dir of Java files
## csvPath: csv Database from project
def extractCode(Outfile,Javadir,csvPath):
    dataframe = pd.read_csv(csvPath, na_filter = False)
    names = ['id', 'type', 'comment', 'code','non-information']
    df = pd.DataFrame()
    for i,data in dataframe.iterrows():
#         print(i)
        link = data["link_to_comment"]
        file_type = data["type"]
        javaFileName = get_java_file_name(link)
        line = get_comment_line(link)

        f = open(Javadir+"/" + javaFileName, "r")
        lines = f.readlines()
        line = line - 1
        code = 'undefined type'
        if file_type == 'Javadoc':
            code = get_code_from_javadoc_comment(lines, line)
        elif file_type == 'Line':
            code = get_code_from_line_comment(lines, line)
        elif file_type == 'Block':
            code = get_code_from_block_comment(lines, line)

    #     print("Code:",code)
    #     print()
        df = df.append(pd.DataFrame([[data["ID"],data["type"],data["comment"],code,data["non-information"]]], columns=names))
        f.close()
#     print(df)
    df.to_csv(Outfile, index=False)

In [6]:
data = dataframe[['type', 'link_to_comment']]
idxx = 0
for row in data.itertuples():
    print(idxx)
    type = row.type
    link = row.link_to_comment
    
    javaFileName = get_java_file_name(link)
    line = get_comment_line(link)
    
    print('type:', type)
    print('Java file:', javaFileName)
    print('Comment start line:', line)
	
    f = open("data/" + javaFileName, "r")
    lines = f.readlines()
    line = line - 1
    code = 'undefined type'
    if type == 'Javadoc':
        code = get_code_from_javadoc_comment(lines, line)
    elif type == 'Line':
        code = get_code_from_line_comment(lines, line)
    elif type == 'Block':
        code = get_code_from_block_comment(lines, line)
    
    print(code)
    print()
    idxx += 1

0
type: Javadoc
Java file: TextInputControlBehavior.java
Comment start line: 151
public static void showContextMenu(TextArea textArea, ContextMenu contextMenu, ContextMenuEvent e) {

1
type: Line
Java file: CellFactory.java
Comment start line: 95
TABLE_ICONS.put(SpecialField.PRINTED, icon);

2
type: Line
Java file: TemporalAccessorPicker.java
Comment start line: 45
BindingsHelper.bindBidirectional(valueProperty(), temporalAccessorValue,

3
type: Javadoc
Java file: JabRefFrame.java
Comment start line: 1102
private boolean confirmClose(BasePanel panel) {

4
type: Block
Java file: IconTheme.java
Comment start line: 227
INTEGRITY_INFO(MaterialDesignIcon.INFORMATION),

5
type: Line
Java file: ExternalFileTypes.java
Comment start line: 228
ExternalFileType toRemove = null;

6
type: Line
Java file: BibtexKeyPatternPanel.java
Comment start line: 137
for (Map.Entry<EntryType, TextField> entry : textFields.entrySet()) {

7
type: Line
Java file: ApplicationInsightsLogEvent.java
Comment start line

public String getIconName() {

118
type: Line
Java file: JabRefGUI.java
Comment start line: 139
toOpenTab.add(pr);

119
type: Line
Java file: JabRefFrame.java
Comment start line: 301
return;

120
type: Line
Java file: BibtexKeyPatternPanel.java
Comment start line: 147
if (!text.trim().isEmpty()) {

121
type: Line
Java file: Bootstrap.java
Comment start line: 211
String pairs[] = null;

122
type: Line
Java file: CaseChangeMenu.java
Comment start line: 25
List<Formatter> caseChangers = new ArrayList<>();

123
type: Line
Java file: ImportAction.java
Comment start line: 93
List<ImportFormatReader.UnknownFormatImport> imports = new ArrayList<>();

124
type: Javadoc
Java file: ViewModelListCellFactory.java
Comment start line: 23
public class ViewModelListCellFactory<T> implements Callback<ListView<T>, ListCell<T>> {

125
type: Line
Java file: GroupNodeViewModel.java
Comment start line: 123
}

126
type: Line
Java file: ImportAction.java
Comment start line: 147
}

127
type: Javadoc
Java file: 

fields.add(InternalField.KEY_FIELD);

255
type: Line
Java file: OOBibBase.java
Comment start line: 462
List<String> tmp = new ArrayList<>();

256
type: Line
Java file: OOBibBase.java
Comment start line: 791
Map<BibEntry, BibDatabase> entries = findCitedEntries(databases, cited, linkSourceBase);

257
type: Line
Java file: ManageKeywordsViewModel.java
Comment start line: 63
for (BibEntry entry : entries) {

258
type: Javadoc
Java file: DefaultTaskExecutor.java
Comment start line: 22
public class DefaultTaskExecutor implements TaskExecutor {

259
type: Line
Java file: BaseDialog.java
Comment start line: 28
if (event.getCode() == KeyCode.ENTER) {

260
type: Line
Java file: OOBibBase.java
Comment start line: 603
refKeys.get(marker).add(currentKey);

261
type: Line
Java file: ExternalFileTypes.java
Comment start line: 221
if (toRemove != null) {

262
type: Javadoc
Java file: JabRefFrame.java
Comment start line: 130
public class JabRefFrame extends BorderPane {

263
type: Line
Java file: Undo

result = OpenDatabase.loadDatabase(diskdb.toAbsolutePath().toString(), importFormatPreferences, Globals.getFileUpdateMonitor());

398
type: Javadoc
Java file: AutoCompletionTextInputBinding.java
Comment start line: 38
public class AutoCompletionTextInputBinding<T> extends AutoCompletionBinding<T> {

399
type: Javadoc
Java file: LinkedFilesEditorViewModel.java
Comment start line: 85
public static LinkedFile fromFile(Path file, List<Path> fileDirectories, ExternalFileTypes externalFileTypesFile) {

400
type: Javadoc
Java file: SuggestionProvider.java
Comment start line: 62
public static <T> SuggestionProvider<T> create(Callback<T, String> stringConverter, Collection<T> possibleSuggestions) {

401
type: Line
Java file: ImportEntriesViewModel.java
Comment start line: 179
entries.remove(entry);

402
type: Javadoc
Java file: MainTable.java
Comment start line: 354
public void updateFont() {

403
type: Javadoc
Java file: UpdateTimestampListener.java
Comment start line: 8
class UpdateTimestampL

public void requestFocus(Field fieldName) {

586
type: Javadoc
Java file: EditorTextArea.java
Comment start line: 49
public void setPasteActionHandler(PasteActionHandler handler) {

587
type: Javadoc
Java file: GroupTreeViewModel.java
Comment start line: 87
private void onSelectedGroupChanged(ObservableList<GroupNodeViewModel> newValue) {

588
type: Line
Java file: ImportAction.java
Comment start line: 137
for (BibEntry entry : pr.getDatabase().getEntries()) {

589
type: Javadoc
Java file: FieldFormatterCleanupsPanel.java
Comment start line: 161
private GridPane getSelectorPanel() {

590
type: Javadoc
Java file: GroupTreeViewModel.java
Comment start line: 117
private void onActiveDatabaseChanged(Optional<BibDatabaseContext> newDatabase) {

591
type: Line
Java file: OSX.java
Comment start line: 29
String[] cmd = (application != null) && !application.isEmpty() ? new String[] {"/usr/bin/open", "-a",

592
type: Line
Java file: GroupDialog.java
Comment start line: 156
keywordGroupSearchFiel

TABLE_ICONS.put(SpecialField.PRIORITY, icon);

744
type: Line
Java file: MergeEntries.java
Comment start line: 315
if (!identicalTypes && !typeRadioButtons.isEmpty() && typeRadioButtons.get(0).isSelected()) {

745
type: Line
Java file: SuggestionProvider.java
Comment start line: 150
if (this.stringConverter == null) {

746
type: Javadoc
Java file: ThemeLoader.java
Comment start line: 76
public void installCss(Scene scene, JabRefPreferences preferences) {

747
type: Line
Java file: GroupNodeViewModel.java
Comment start line: 272


748
type: Line
Java file: FileFilterConverter.java
Comment start line: 71
return pathname -> true;

749
type: Javadoc
Java file: VersionWorker.java
Comment start line: 30
private final Version installedVersion;

750
type: Line
Java file: URLUtil.java
Comment start line: 112
if (strippedLink.substring(strippedLinkIndex + 1).indexOf('/') >= 1) {

751
type: Javadoc
Java file: JabRefFrame.java
Comment start line: 916
public void setupAllTables() {

752
type: Line


866
type: Line
Java file: ArgumentProcessor.java
Comment start line: 255
if (!matches.isEmpty()) {

867
type: Line
Java file: ArgumentProcessor.java
Comment start line: 259
switch (data.length) {

868
type: Line
Java file: ArgumentProcessor.java
Comment start line: 265
formatName = "tablerefsabsbib";

869
type: Line
Java file: ArgumentProcessor.java
Comment start line: 275
Optional<Exporter> exporter = Globals.exportFactory.getExporterByName(formatName);

870
type: Line
Java file: ArgumentProcessor.java
Comment start line: 318
boolean bibExtension = aLeftOver.toLowerCase(Locale.ENGLISH).endsWith("bib");

871
type: Line
Java file: ArgumentProcessor.java
Comment start line: 328
if (startupMode == Mode.INITIAL_START) {

872
type: Line
Java file: ArgumentProcessor.java
Comment start line: 372
if ((newBase != null) && newBase.hasEntries()) {

873
type: Line
Java file: ArgumentProcessor.java
Comment start line: 397
if (fileWriter.hasEncodingProblems()) {

874
type: Line
Java file: ArgumentPr



1027
type: Line
Java file: OracleConnection.java
Comment start line: 221


1028
type: Line
Java file: OracleConnection.java
Comment start line: 228
return null;

1029
type: Line
Java file: OracleConnection.java
Comment start line: 235
return null;

1030
type: Line
Java file: OracleConnection.java
Comment start line: 242
return null;

1031
type: Line
Java file: OracleConnection.java
Comment start line: 248
return null;

1032
type: Line
Java file: OracleConnection.java
Comment start line: 254
return null;

1033
type: Line
Java file: OracleConnection.java
Comment start line: 260
return null;

1034
type: Line
Java file: OracleConnection.java
Comment start line: 266
return null;

1035
type: Line
Java file: OracleConnection.java
Comment start line: 272
return null;

1036
type: Line
Java file: OracleConnection.java
Comment start line: 278
return null;

1037
type: Line
Java file: OracleConnection.java
Comment start line: 284
return null;

1038
type: Line
Java file: OracleConnection.java
Comm

differences.add(new BibStringDiff(original, match.get()));

1132
type: Line
Java file: BibStringDiff.java
Comment start line: 48
notMatched.add(original);

1133
type: Line
Java file: BibStringDiff.java
Comment start line: 53
for (Iterator<BibtexString> iterator = notMatched.iterator(); iterator.hasNext(); ) {

1134
type: Line
Java file: BibStringDiff.java
Comment start line: 74
newDatabase.getStringValues().stream()

1135
type: Javadoc
Java file: BibtexCaseChanger.java
Comment start line: 129
private int convertSpecialChar(StringBuilder sb, char[] c, int start, FORMAT_MODE format) {

1136
type: Line
Java file: ACS.java
Comment start line: 47
Document html = Jsoup.connect(source).ignoreHttpErrors(true).get();

1137
type: Javadoc
Java file: BibtexCaseChanger.java
Comment start line: 265
public static Optional<String> findSpecialChar(char[] c, int pos) {

1138
type: Line
Java file: BibtexCaseChanger.java
Comment start line: 23
ALL_LOWERS('l'),

1139
type: Block
Java file: BibDatabaseWrite

PICTURE(MaterialDesignIcon.FILE_IMAGE)

1284
type: Line
Java file: JabRefFrame.java
Comment start line: 1241
TextInputControl textInput = (TextInputControl) focusOwner;

1285
type: Block
Java file: IconTheme.java
Comment start line: 230
GITHUB(MaterialDesignIcon.GITHUB_CIRCLE),

1286
type: Block
Java file: IconTheme.java
Comment start line: 231
TOGGLE_ENTRY_PREVIEW(MaterialDesignIcon.LIBRARY_BOOKS),

1287
type: Block
Java file: IconTheme.java
Comment start line: 250
CHECK(MaterialDesignIcon.CHECK)

1288
type: Block
Java file: IconTheme.java
Comment start line: 254
REG_EX(MaterialDesignIcon.REGEX),

1289
type: Line
Java file: JabRefFrame.java
Comment start line: 1266
break;

1290
type: Line
Java file: ParserResultWarningDialog.java
Comment start line: 69
jabRefFrame.getDialogService().showWarningDialogAndWait(dialogTitle, dialogContent.toString());

1291
type: Line
Java file: ImportAction.java
Comment start line: 100
imports.add(Globals.IMPORT_FORMAT_READER.importUnknownFormat(filename,

In [7]:
Outfile ="csv/oneLineCode2.csv"
Javadir ="data/"
csvPath = "assets/train_set_0520.csv"
extractCode(Outfile,Javadir,csvPath)

In [8]:
csvPath = "csv/oneLineCode2.csv"
dataframe = pd.read_csv(csvPath, na_filter = False)
keep_col = ['comment','code','non-information']
new_f = dataframe[keep_col]
new_f.to_csv("csv/newFile2.csv", index=False)
